## init

In [ ]:
%pip install -qqq transformers torchaudio datasets

In [ ]:
from google.colab import drive
import os

In [ ]:
DRIVE_MOUNT_PATH = "/content/drive"
DATA_PATH = f"{DRIVE_MOUNT_PATH}/MyDrive/Shared/data"

In [ ]:
drive.mount(DRIVE_MOUNT_PATH)

# **Use wav2vec2 for speech emotion recognition on IEMOCAP dataset**
---
- 🚀 **objective**: run wav2vec2 as a feature extractor on IEMOCAP dataset, requires the data preprocessing of IEMOCAP dataset  
- 🧯 **models**: wav2vec2
- 📚 **dataset**: IEMOCAP


Resources
- inspired by https://colab.research.google.com/github/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb#scrollTo=Fv62ShDsH5DZ

## ⚙️ configuration

In [ ]:
import numpy as np

from transformers import TrainingArguments
import torch

In [ ]:
model_name_or_path = "facebook/wav2vec2-large-960h"
feature_to_idx = {key: i for i, key in enumerate(["wav", "sampling_rate", "filename", "label", "speaker"])}
label_list = ["neu", "hap", "ang", "sad", "exc", "fru"]
num_labels = len(label_list)
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

pooling_mode = "max"
test_split_size = 0.2
target_sampling_rate = 16000

DEBUG_SIZE = 10

In [ ]:
# training parameters
training_args = TrainingArguments(
    output_dir="/content/wav2vec2-iemocap-speech-emotion-recognition",
    label_names=label_list,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=torch.cuda.is_available(),
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)

## 📚 data
- ~torchaudio implemented a `dataset` to load IEMOCAP. Later in the script, we train the model with a `Trainer` from hugginface, therefore we prefer translating the pytorch dataset into a `transformers.Dataset` for convenience and compatibility.~
- the Trainer class expects an argument `train_dataset` to be of type torch.utils.data.Dataset (see [documentation](https://huggingface.co/docs/transformers/main_classes/trainer)) --> we use a torch dataset instead of a Hugginface dataset

In [ ]:
# https://pytorch.org/audio/master/generated/torchaudio.datasets.IEMOCAP.html
from torchaudio.datasets import IEMOCAP

from transformers import Wav2Vec2Processor

import torch
from torch.utils.data import random_split, Dataset, DataLoader, SubsetRandomSampler

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
target_sampling_rate = processor.feature_extractor.sampling_rate

In [ ]:
class CustomIEMOCAP(Dataset):
  def __init__(self, data, processor):
    self.data = data
    self.processor = processor

  def __getitem__(self, index):
    wav, _, _, label, _ = self.data[index]
    inputs = self.processor(wav.squeeze(), sampling_rate=target_sampling_rate)
    inputs["labels"] = label2id[label]

    return inputs

  def __len__(self):
    return len(self.data)

In [ ]:
iemocap = IEMOCAP(root=DATA_PATH) # in function, path = root / "IEMOCAP"
dataset = CustomIEMOCAP(data=iemocap, processor=processor)
train_ds, test_ds = random_split(dataset, [test_split_size, 1-test_split_size], generator=torch.Generator().manual_seed(42))

dataset[0]

In [ ]:
# # generate a huggingface dataset
# iemocap = IEMOCAP(root=DATA_PATH) # in function, path = root / "IEMOCAP"

# def _generate_dicts(ds: IEMOCAP):
#   for i in range(min(DEBUG_SIZE, len(ds))):
#     wav_path, sample_rate, filename, label, speaker = ds.get_metadata(i)
#     yield {
#         "audio": DATA_PATH + '/IEMOCAP/' + wav_path,
#         "label": label, 
#         "speaker": speaker
#     }
    

# dataset = Dataset.from_generator(lambda: _generate_dicts(iemocap)).cast_column("audio", Audio(sampling_rate=target_sampling_rate))
# label_list = dataset.unique('label')
# num_labels = len(label_list)

# split_dataset = dataset.train_test_split(test_size=test_split_size)
# train_ds, test_ds = split_dataset['train'], split_dataset['test']

# print(dataset)
# print(train_ds)
# print(test_ds)

## 🚜 model definition

In [ ]:
from transformers import Wav2Vec2Model, Wav2Vec2PreTrainedModel
from transformers import AutoConfig

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

In [ ]:
# model configuration
config = AutoConfig.from_pretrained(
  model_name_or_path,
  num_labels=num_labels,
  label2id=label2id,
  id2label=id2label,
  finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)
config

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Tuple
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
class Wav2Vec2ClassificationHead(nn.Module):
  """Head for wav2vec classification task."""

  def __init__(self, config):
    super().__init__()
    self.dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.dropout = nn.Dropout(config.final_dropout)
    self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self, features, **kwargs):
    x = features
    x = self.dropout(x)
    x = self.dense(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.out_proj(x)
    return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    self.pooling_mode = config.pooling_mode
    self.config = config

    self.wav2vec2 = Wav2Vec2Model(config)
    self.classifier = Wav2Vec2ClassificationHead(config)

    self.init_weights()

  def freeze_feature_extractor(self):
    self.wav2vec2.feature_extractor._freeze_parameters()

  def merged_strategy(
      
      self,
      hidden_states,
      mode="mean"
  ):
    if mode == "mean":
        outputs = torch.mean(hidden_states, dim=1)
    elif mode == "sum":
        outputs = torch.sum(hidden_states, dim=1)
    elif mode == "max":
        outputs = torch.max(hidden_states, dim=1)[0]
    else:
        raise Exception(
            "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

    return outputs

  def forward(
      self,
      input_values,
      attention_mask=None,
      output_attentions=None,
      output_hidden_states=None,
      return_dict=None,
      labels=None,
  ):
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    outputs = self.wav2vec2(
        input_values,
        attention_mask=attention_mask,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )
    hidden_states = outputs[0]
    hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
    logits = self.classifier(hidden_states)

    loss = None
    if labels is not None:
      loss_fct = CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    if not return_dict:
      output = (logits,) + outputs[2:]
      return ((loss,) + output) if loss is not None else output

    return SpeechClassifierOutput(
      loss=loss,
      logits=logits,
      hidden_states=outputs.hidden_states,
      attentions=outputs.attentions,
    )

In [ ]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

In [ ]:
model.freeze_feature_extractor()

## 🏃‍♀️ training routine

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, Union, Tuple, Optional
from packaging import version
import numpy as np

import torch
from torch import nn

from transformers import Trainer, is_apex_available, Wav2Vec2Processor, EvalPrediction, ProcessorMixin

if is_apex_available():
    from apex import amp 
    # Apex is a PyTorch add-on package from NVIDIA with capabilities for automatic mixed precision (AMP) and distributed training.
    # https://www.ibm.com/docs/en/wmlce/1.6.1?topic=frameworks-getting-started-apex

if version.parse(torch.__version__) >= version.parse("1.6"):
  _is_native_amp_available = True
  from torch.cuda.amp import autocast

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
  processor: Wav2Vec2Processor
  padding: Union[bool, str] = True
  max_length: Optional[int] = None
  max_length_labels: Optional[int] = None
  pad_to_multiple_of: Optional[int] = None
  pad_to_multiple_of_labels: Optional[int] = None

  def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
    input_features = [{"input_values": feature["input_values"][0]} for feature in features]
    label_features = [feature["labels"] for feature in features]

    d_type = torch.long if isinstance(label_features[0], int) else torch.float

    batch = self.processor.pad(
      input_features,
      padding=self.padding,
      max_length=self.max_length,
      pad_to_multiple_of=self.pad_to_multiple_of,
      return_tensors="pt",
    )

    batch["labels"] = torch.tensor(label_features, dtype=d_type)
    # print('batch', batch)
    return batch

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
class CTCTrainer(Trainer):
  def training_step(self, processor: ProcessorMixin, inputs) -> torch.Tensor:
    self.model.train()
    inputs = self._prepare_inputs(inputs)

    loss = self.compute_loss(self.model, inputs)

    if self.args.gradient_accumulation_steps > 1:
      loss = loss / self.args.gradient_accumulation_steps

    self.scaler.scale(loss).backward()

    return loss.detach()

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from transformers import DataCollatorWithPadding
trainer = CTCTrainer(
  model=model,
  data_collator=data_collator,
  args=training_args,
  compute_metrics=compute_metrics,
  train_dataset=train_ds,
  eval_dataset=test_ds,
  tokenizer=processor.feature_extractor,
)

## 🧪 experiments

In [ ]:
WANDB_DISABLED = "false"
trainer.train()